# Импорт

In [1]:
!pip install catboost
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c6abc17df2a8b8e122fbf4db9785e84a72cbf4d0e6ccb392d3c2bffba17d0ef4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
# при работе через Google Colab
# прописать путь до всех файлов py
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Учёба/8 семестр/диплом/Проект/VKR/Блокноты')

Mounted at /content/drive


In [3]:
import AnalysisFunc
import BaseFuncs
import PredictionSplit
import PredictionResults

ДЛЯ ПРОГНОЗИРОВАНИЙ ДОСТУПНЫ СЛЕДУЮЩИЕ ПОДСЧЕТЫ ОШИБОК/ТОЧНОСТЕЙ:
recall
f1
r2
mae


In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, confusion_matrix, f1_score, classification_report, recall_score, mean_absolute_percentage_error
# from sklearn import preprocessing

import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


# Загрузка данных

In [18]:
from google.colab import drive
drive.mount('/content/drive')
data = BaseFuncs.GetDataFrameListFromFileConverted("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Преобразованные/ПиОА 1 div20kr.csv")
data = AnalysisFunc.MakeFloat(data, ["Пол"])
dataReal = BaseFuncs.GetDataFrameListFromFileConverted("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Преобразованные/ПиОА тест div20kr.csv")
dataReal = AnalysisFunc.MakeFloat(dataReal, ["Пол"])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# загрузка данных через виндовс
data = BaseFuncs.GetDataFrameListFromFileConverted(BaseFuncs.OpenFileNameWindows())


In [6]:
data.drop_duplicates(keep='last')

ex = "Контрольная работа 2"

# Словари параметров

In [19]:
parameters_AdaBoostRegressor = {
     'n_estimators': range(1, 15),
    'learning_rate' : [0.01, 0.05, 0.1, 0.5],
    'loss' : ['linear', 'square', 'exponential'],
}

parameters_RandomForestRegressor = {'bootstrap': [True], 'max_depth': range(1, 15),
                'max_features': ['sqrt', 'log2', None],
                'min_samples_leaf': [1,2,3],
                'min_samples_split': [1,2,3],
                'n_estimators': range(1, 15)}

parameters_DecisionTreeRegressor={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': range(1, 15),
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1,2,3],
                         'min_samples_split': [1,2,3],
                         'splitter': ['best', 'random']}

parameters_KNeighborsRegressor = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}

parameters_CatBoostRegressor = {
    'iterations': range(1,251, 25),
    'silent': [True],
    'depth': range(1, 10, 1),
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1]
}



parameters_AdaBoostClassifier = {
    'n_estimators':range(1,50),
    'base_estimator':[None, DecisionTreeClassifier(), SVC(), LogisticRegression()],
    'learning_rate': [0.5, 1.0, 2.0, ]
}

parameters_RandomForestClassifier={'max_depth':(1,15, 1),
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }
parameters_DecisionTreeClassifier={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 15),
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1,2,3],
                         'min_samples_split': [1,2,3],
                         'splitter': ['best', 'random']}

parameters_KNeighborsClassifier = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}


parameters_CatBoostClassifier = {
    'iterations': range(1,251, 10),
    'silent': [True],
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1, 0.5]
}

parameters_sgd = {
    'loss':['log'],
    'penalty':['elasticnet'],
    'n_iter_no_change':[5],
    'alpha':np.logspace(-4, 4, 10),
    'l1_ratio':[0.05,0.06,0.07,0.08,0.09,0.1,0.12,0.13,0.14,0.15,0.2]
}

# Определение параметров контрольной



In [ ]:
#  найти параметры регрессии
X, Y = PredictionSplit.SplitTestTrainTest(data, ex, ["Пол", "Оценка"]+AnalysisFunc.before)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)


grid_search=GridSearchCV(cv=5, estimator=AdaBoostRegressor(), n_jobs=-1,
             param_grid=parameters_AdaBoostRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"AdaBoostRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )


grid_search=GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid=parameters_RandomForestRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"RandomForestRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )



grid_search=GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid=parameters_DecisionTreeRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"DecisionTreeRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20)



grid_search=GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid=parameters_KNeighborsRegressor,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"KNeighborsRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20)



grid_search = GridSearchCV(
    estimator = CatBoostRegressor(),
    param_grid = parameters_CatBoostRegressor,
    scoring = 'neg_mean_absolute_error',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"CatBoostRegressor best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_*20 )

Fitting 5 folds for each of 168 candidates, totalling 840 fits
AdaBoostRegressor parameters: {'learning_rate': 0.01, 'loss': 'square', 'n_estimators': 12}
Best Score: -3.453942699586733
Fitting 5 folds for each of 5292 candidates, totalling 26460 fits
RandomForestRegressor parameters: {'bootstrap': True, 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 6}
Best Score: -3.400402267367352
Fitting 5 folds for each of 3024 candidates, totalling 15120 fits
DecisionTreeRegressor parameters: {'criterion': 'absolute_error', 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter': 'random'}
Best Score: -3.4058494195688227
Fitting 5 folds for each of 2400 candidates, totalling 12000 fits
KNeighborsRegressor parameters: {'leaf_size': 34, 'metric': 'minkowski', 'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}
Best Score: -3.461290744689252


# Определение параметров сдачи

In [20]:
#найти наибольшую глубину
def FindParametersClasses(X_train, y_train, X_test, y_test, error_type):
    grid_search = GridSearchCV( estimator=AdaBoostClassifier(), param_grid=parameters_AdaBoostClassifier, scoring = error_type,
        n_jobs = -1,
        cv = 5)
    pred = grid_search.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print(f"AdaBoostClassifier parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )



    grid_search = GridSearchCV(
        estimator = RandomForestClassifier(),
        param_grid = parameters_RandomForestClassifier,
        scoring =error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print(f"RandomForestClassifier parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )

    grid_search = GridSearchCV(
        estimator = DecisionTreeClassifier(),
        param_grid = parameters_DecisionTreeClassifier,
        scoring =error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred = pred.predict(X_test)
    print(f"DecisionTreeClassifier parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )


    grid_search = GridSearchCV(
        estimator = KNeighborsClassifier(),
        param_grid = parameters_KNeighborsClassifier,
        scoring = error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred_KNN1 =pred.predict(X_test)
    print(f"Knn best parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )



    grid_search = GridSearchCV(
        estimator = CatBoostClassifier(),
        param_grid = parameters_CatBoostClassifier,
        scoring = error_type,
        n_jobs = -1,
        cv = 5
    )
    pred=grid_search.fit(X_train, y_train)
    y_pred_KNN1 =pred.predict(X_test)
    print(f"CatBoostClassifier best parameters: {grid_search.best_params_}" )
    print('Best Score:', grid_search.best_score_ )


In [21]:
X, Y, = PredictionSplit.SplitTestTrainPass(data, ex, ["Пол", "Оценка", "Направление"]+AnalysisFunc.before)
x_, y_ = PredictionSplit.SplitTestTrainPass(dataReal, ex, ["Пол", "Оценка", "Направление"]+AnalysisFunc.before)

In [ ]:
FindParametersClasses(X, Y, x_, y_, 'recall')

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)
grid_search = GridSearchCV(
        estimator = SGDClassifier(),
        param_grid = parameters_sgd,
        scoring = "precision",
        n_jobs = -1,
        cv = 5
    )
pred=grid_search.fit(x_, y_)
y_pred_KNN1 =pred.predict(X)
print(f"SGDClassifier best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )

SGDClassifier best parameters: {'alpha': 0.046415888336127774, 'l1_ratio': 0.2, 'loss': 'log', 'n_iter_no_change': 5, 'penalty': 'elasticnet'}
Best Score: 0.9941176470588236


# Определение параметров контрольной как классов

In [ ]:
#найти наибольшую глубину
X, Y = PredictionSplit.SplitTestTrainTest(data, ex, ["Пол", "Оценка"])
Y = Y*20
Y = Y.round(0)
FindParametersClasses(X, Y, 'neg_mean_absolute_error')

AdaBoostClassifier parameters: {'base_estimator': LogisticRegression(), 'learning_rate': 1.0, 'n_estimators': 45}
Best Score: -3.8732891100055284
RandomForestClassifier parameters: {'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 100}
Best Score: -3.76057490326147
DecisionTreeClassifier parameters: {'criterion': 'log_loss', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 3, 'splitter': 'random'}
Best Score: -3.883095632946379
Knn best parameters: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 8, 'p': 1, 'weights': 'distance'}
Best Score: -3.634118297401879
CatBoostClassifier best parameters: {'allow_writing_files': False, 'iterations': 181, 'learning_rate': 0.05, 'silent': True}
Best Score: -3.5805638474295187
